If you're running this in Google Colab, you can click "Copy to Drive" (above &#8593;) or go to **File > Save a Copy in Drive** so you'll have your own version to work on. That requires a Google login.  
<hr/>

# Lab 3: Practice Fitting a Model   
This Jupyter notebook is a template for fitting a model function to a data set and comparing which models fit better. If you've ever used a trendline, that's an example of a model. Python is good for doing that with all sorts of functions, not just lines, and will allow you to later analyze data sets too big for a spreadsheet.      

## Instructions  
Run each block of code below and discuss the self-check questions at the end of each **Part**. You won't need to record your answers to most of those questions, just the items called for in the lab requirements given at the very end. If you need to start over from scratch, open a [clean copy of this activity](https://colab.research.google.com/github/adamlamee/UCF_labs/blob/main/model_fitting_practice.ipynb). If you'd like a refresher on how to work with Python notebooks, try the [intro activity](https://colab.research.google.com/github/adamlamee/UCF_labs/blob/main/intro.ipynb).  
- **Groups of 2**: Take turns with who does the typing and swap after each **Part** below while the other member looks for errors and makes suggestions.  
- **Groups of 3**: Take turns with who does the typing and swap after each **Part** below while the other two members look for errors and make suggestions.  
- **Groups of 4**: Split into two pairs to do the coding below, but come back together to discuss each set of self-check questions. In each pair, take turns with who does the typing and swap after each **Part** below while the other member makes suggestions and looks for errors.  
- **Individuals**: You're welcome to do this lab on your own without penalty. Do not add yourself to a group of one in Webcourses. Just submit this lab on your own.  

## Part 1: Examine a data set  
This notebook starts with reading in a set of time and position measurements for an object speeding up (that's already provided for you). 

In [ ]:
# import modules
import pandas as pd    # pandas has functions to read and manipulate data
import numpy as np     # numpy has functions to do caluclations
import matplotlib.pyplot as plt    # pyplot has functions to make plots
from scipy.optimize import curve_fit  # scipy has functions for model fitting
import warnings
warnings.filterwarnings('ignore')    # doesn't distract with warnings, still shows useful error messages

In [ ]:
# read in data
data = pd.read_csv('https://github.com/adamlamee/UCF_labs/raw/main/data/motion.csv')

In [ ]:
# shows the size of the data set with the number of (rows, columns)
data.shape

In [ ]:
# view the column headings and first few rows of the data set
data.head(3)    # you can change the number in the ()

In [ ]:
# view *last few rows of the data set
data.tail(3)    # you can change the number in the ()

In [ ]:
# shows the column headings in the file
# it can be useful to copy directly from here to get the 'exact' column name
data.columns

In [ ]:
# a scatterplot fo the entire data set can help get an idea of what we're deadling with
# scatterplots take two columns (x, then y) follwed by additional parameters you can set
plt.scatter(data['Time (s)'], data['Position (m)'], s=.1, color='darkorchid')
plt.xlabel("label me")
plt.ylabel("?")
plt.title("what?")
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

### Self-Check for Part 1  
This is a good place to pause and ask yourself some questions about the code above.  
- How many position measurements are there?  
- What was the duration of this data collection?  
- Can you set better limits for the axes? (It's good practice to start the y axis at zero) 
- Can you set a better title and axis labels?  
- What might be a good type of function to fit to this data?  
- Can you change the color of the scatterplot? Here's a [longer list of color names](https://matplotlib.org/stable/gallery/color/named_colors.html).  
- Can you figure out what the 's' parameter sets in the plt.scatter function?  

Here are the [answers to the self-check questions](https://github.com/adamlamee/UCF_labs/blob/main/self_check_model_fitting.md).  

## Part 2: Fit a trendline  
You've already used several Python functions, like [Pandas' read_csv( )](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) and [PyPlot's scatter( )](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html). Next, you'll define your own function. That function will take x-values (time) as inputs and output model y-values (position) that are close to the measured y-values. Then, you'll use the [SciPy function curve_fit( )](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to determine the best coefficients for your model equation. SciPy is short for *scientific python* and regrettably is pronounced 'psi-pi' and *not* 'skippy'.  

In [ ]:
# define the function you'll fit
def model_1(x, a, b):   # first argument (x) is input data, followed by coefficients of the model function
    y = a * x + b        # that's just a line equation
    return(y)

# the function curve_fit() takes three arguments: (model function, x-values, y-values)
# then calculates the optimized coefficients for your model function (called popt)
popt, pcov = curve_fit(model_1, data['Time (s)'], data['Position (m)'])

# makes a new column and fills it with values using the 'my_model' function above
data['model_1'] = model_1(data['Time (s)'], *popt)

# look at the first few rows to see if the code did what we wanted it to
data.head(3)

In [ ]:
# outputs the optimized model coefficients
popt   # these are in the order they appear in your model function .. slope (a) and y-int (b) in the example

In [ ]:
# makes a scatterplot of the measured data the model's y-values
plt.scatter(data['Time (s)'], data['Position (m)'], s=.1, color='darkorchid')    # plots time (x) and temp (y)
plt.scatter(data['Time (s)'], data['model_1'], s=.1, color='goldenrod')  # plots time (x) and model values (y)
plt.show()

### Self-Check for Part 2  
- What's the name of the function you defined to calculate the model y-values?  FYI: you can rename it, but you'll need to update that name in the code that follows it to fill the new column.  
- What is the line equation for your trendline, including the optimized coefficients?  
- How well does a line fit this data? What are the signs that this data isn't quite linear? Can you think of any functions that might be a better fit?  
- Can you set the labels and axis limits for this new scatterplot? A common practice is to copy/paste from other working code instead of writing everyting from scratch.  

Here are the [answers to the self-check questions](https://github.com/adamlamee/UCF_labs/blob/main/self_check_model_fitting.md).  

## Part 3: Evaluate the Model Function's Fit
One way to quantify how well a model function fits the measured data is to analyze its *residuals*. That's the vertical distance between the trendline (or curve) and each point on the scatterplot. Squaring each residual and adding them up is the *residual sum of squares* for the model, or RSS.  

In [ ]:
# calculate residuals
data['res_1'] = data['Position (m)'] - data['model_1']

# look at the first few rows to see if the code did what we wanted it to
data.head(3)

In [ ]:
# plot residuals vs the x-variable. That's 'time' in this example
plt.scatter(data['Time (s)'], data['res_1'], s=.5, color='green')
plt.axhline(y=0, color='grey')          # draws the x-axis at zero to help visualize
plt.xlabel("label me")
plt.ylabel("")
plt.title("")
plt.show()

In [ ]:
# makes a new column of squared residuals
data['res_1_squared'] = data['res_1'] ** 2

# look at the first few rows to see if the code did what we wanted it to
data.head(3)

In [ ]:
# outputs the sum of the squared residuals column ... that's the RSS for this model
data['res_1_squared'].sum()

### Self-Check for Part 3  
- Could a residual be zero? Could it be negative?  
- If all the data points fell exactly on the trendline, what would the model's RSS be? Could a model's RSS be negative?  
- What is this model's RSS?  
- While the RSS for a single model doesn't tell us much, how could comparing the RSS values for two models help determine which one fits the data better?  
- If the position data were linear, the residuals would be evenly distributed across the residual plot. However, the residuals in the middle of the plot are mostly while the residuals at the left and right are positive. What does that communicate about this model function?  
- What shape of model function might fit this data better?  
- Whenever possible, you should choose a model function that has some basis in theoritical predicitons related to the data. Can you think of any functions that might be reasonable to describe the position of an object speeding up?  

Here are the [answers to the self-check questions](https://github.com/adamlamee/UCF_labs/blob/main/self_check_model_fitting.md).  

## Part 4: Fit a Second Model  
Now that you've fit a linear model function to the motion data you can see a curve might do a better job at fitting the data. This time, use a function with three coefficients (a, b, & c) to help improve the fit.  Note that SciPy curve_fit isn't always able to optimize a model so you may get a very poor fit (or error message) even when you have a function you think should fit well. There are other fitting methods in Python to get around this, but we'll save that for another time.  

Here are some ideas of functions to use for your new model or you're welcome to try something else. Beyond polynomials, you'll need to use a NumPy function (alas, it's 'num pie' not 'numpee'). You can find a full list of [NumPy functions here](https://numpy.org/doc/stable/reference/routines.math.html). Note you'll need to start each one with "np." to tell the code it's in the NumPy module.    

| Some example functions | Use this when defining your model |  
| :-: | :-: |  
| sin(x) | y = a * np.sin(b * x) + c |  
| quadratic | y = a * x ** 2 + b * x + c |  
| natural log, ln | y = a * np.log(b * x) + c |  
| exponential decay, e<sup>-x</sup> | y = a - b * np.exp(-c * x) |   


In [ ]:
# define a new model function called model_2

def model_2(x, a, b, c):       # note: this now takes three coefficients a, b, & c
    y = ?         # <--- edit this to write a new model function  !!!!!!!!!!!!!!!
    return(y)

# use the curve_fit() function to calculate optimized coefficients for your new model
popt, pcov = curve_fit(model_2, data['Time (s)'], data['Position (m)']) # note this now says "model_2"

# make a new column to store the y-values for your new model
data['model_2'] = model_2(data['Time (s)'], *popt)    # note this now says "model_2"

# look at the first few rows to see if the code did what we wanted it to
data.head(3)

In [ ]:
# output popt again to view the optimized parameters (i.e., coefficients) for your new model

popt

In [ ]:
# make a scatterplot of the measured data and your new model's y-values (instead of the trendline)

############################
# copy/paste from Part 2 and edit the code to analyze your new model
############################

In [ ]:
# calculate the residuals for the new model and give that column a different name than 'res_1'

############################
# copy/paste from Part 3 and edit the code to analyze your new model
############################

# use the .head() function to view your data table and see if your code did what you wanted it to

data.head(3)

In [ ]:
# make a residual plot for this model

############################
# copy/paste from Part 3 and edit the code to analyze your new model
############################

In [ ]:
# add a column for residuals squared and give it a different name than 'res_1_squared'

############################
# copy/paste from Part 3 and edit the code to analyze your new model
############################

# use the .head() function to view your data table and see if your code did what you wanted it to

data.head(3)

In [ ]:
# calculate the RSS for the new model

############################
# copy/paste from Part 3 and edit the code to analyze your new model
############################

### Self-Check for Part 4  
- How well does your new model appear to fit the motion data in the scatterplot?  
- Does the residual plot show any pattern like it did for the trendline? 
- What is your new model function equation, including the optimized coefficients?  
- Based on the RSS, did your new model fit the data better than the trendline?  
- If another group were going to try fitting a different function, how would you advise them?  

Here are the [answers to the self-check questions](https://github.com/adamlamee/UCF_labs/blob/main/self_check_model_fitting.md).  

## Submission for Lab 3  
Now that you've practiced fitting two different models to the sample motion data, you'll need to choose a different data set and perform a similar analysis. A common practice when repeating a similar analysis is to make a copy of your working Python notebook and just edit the lines that reference the data URL, column headings, etc.  
- Choose another data set from the list below. Then, pick two columns in the new data set you think might be related and repeat this model-fitting analysis.  
- Fit a linear function and one other model function.  
- Submit a pdf containing the following. It can be a pdf export of a Word/Pages/Google Doc or save your Python notebook as a pdf:  
    - (5 pts) each group member's name and project management role  
    - (5 pts) a WBS showing which members did the typing for which parts of the assignment
    - (10 pts) a scatterplot and residual plot for each of the two models for your new data set  
    - (10 pts) your two model equations, with optimized coefficients, and 1-3 sentences describing why you chose your second model fucntion  
    - (10 pts) the RSS values for your two models and a description of which model was the better fit  

## Other Data To Analyze  
To analyze a data file below, copy its "data URL" link and paste it into the pd.read_csv( ) line near the top of this notebook. You'll need to keep the parentheses and single-quotes around the URL in the read function.  
- STRATA-2P ([data URL](https://github.com/adamlamee/UCF_labs/raw/main/data/STRATA-2P_cooling.csv)) is from UCF's STRATA-2P experiment led by UCF's [Dr. Addie Dove](https://planets.ucf.edu/people/faculty/adrienne-dove/) and describes the temperature of lunar regolith simulant (i.e., lab-made moon dust) just after it was heated up in a lunar gravity environment on a parabolic flight. Cool, right?
- Palmer Penguins ([data URL](https://github.com/adamlamee/UCF_labs/raw/main/data/penguins.csv)) describes meaurements made form penguins in Antarctica. The data is actually a subset of [Dr. Allison Horst's]([https://allisonhorst.com/](https://allisonhorst.github.io/)) [Palmer Penguins data](https://allisonhorst.github.io/palmerpenguins/), converted to a [Python-friendly format](https://github.com/mcnakhaee/palmerpenguins) by [Muhammad Chenariyan Nakhaee](https://github.com/mcnakhaee).  
- USGS seismic activity ([data URL]('http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')) is a record of global seismic events for the past month, updated every minute, from the US Geological Survey's [Earthquake hazards program](https://earthquake.usgs.gov/earthquakes/feed/v1.0/csv.php).   
- Periodic table of the elements ([data URL]('http://github.com/adamlamee/CODINGinK12/raw/master/data/elements.csv')) came from Penn State CS professor Doug Hogan, but swapping the signs of the electron affinity values.  

<hr/>  

# Credits
This notebook was written by [Adam LaMee](http://www.adamlamee.com). Thanks to the great folks at [Binder](https://mybinder.org/) and [Google Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb) for making this notebook interactive without you needing to download it or install [Jupyter](https://jupyter.org/) on your own device.  